In [117]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import re

from tqdm import tqdm
from bs4 import BeautifulSoup
from utils import *
from row_utils import *
from string_utils import *

# Set directory & name of save file
results = []
save_file_name = "check1_df"

path_prefix = "./Check_20190717/"
all_2004 = os.listdir(path_prefix)
all_2004 = [x for x in all_2004 if ('DS_Store' not in x)]
all_2004 = [x for x in all_2004 if ('xlsx' not in x)]

# Recording which files have been scraped 
cik_done_dict = get_cik_done(save_file_name)
#cik_done_dict = {}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [125]:
# Scraping the remaining files
results = []
for i in tqdm(range(len(all_2004))):
    
    fname = all_2004[i]
    test_f = open(path_prefix + fname)
    wholetext = test_f.read()
    cikbase = re.search(r"CENTRAL INDEX KEY:\s*?\d{10}\b", wholetext, re.I).group()
    cik_re = re.split(r'\s', cikbase)[-1]        

    datadatebase = re.search(r"CONFORMED PERIOD OF REPORT:\s*?\d{8}\b", wholetext, re.I).group() 
    datadate_re = re.split(r'\s', datadatebase)[-1] 
    if int(cik_re) in cik_done_dict:
        if int(datadate_re) in cik_done_dict[int(cik_re)]:
            print("Already processed this file")
            continue
    
    wholetext = wholetext.lower()
    tbl_flag = 0 # 1 if "contractual obligations" table exists
    row_flag = 0 # 1 if "purchasing obligations" row exists
    multiplier = 1 # 1 by default
    
    table = scrape_table(wholetext)
    rows = []
    if (not table) or ('-----' in table.text):
        multiplier, lines, tbl_list = scrape_text_table(wholetext)
        if tbl_list != None:
            tbl_flag = 1
            rows = scrape_text_rows(lines, tbl_list)
    else:
        tbl_flag = 1
        rows = scrape_rows(table)
        multiplier = get_multiplier_from_tbl(table)
        
    for row in rows:
        row_flag = 1
        
        continue_flag = 0
        all_entries = [float(row['<1']), float(row['1-3']), float(row['3-5']), float(row['>5'])]
        if np.sum(all_entries) == 0:
            continue_flag = 1
        if continue_flag == 1:
            continue
            
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 
                  'row': row_flag, '<1': row['<1'], '1-3':row['1-3'], 
                  '3-5':row['3-5'], '>5':row['>5'], 
                  'category': clean_category_name(row['category']), 
                  'multiplier': multiplier}
        if 'total' in row.keys():
                result['total'] = row['total']
        results.append(result)

    # No table or no row
    if (tbl_flag == 0) or (row_flag == 0):
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                  'total': '-', '<1': '-', '1-3':'-', '3-5':'-', '>5':'-', 'multiplier': multiplier, 'category': 'None'}

        results.append(result)
    pd.DataFrame(results).to_csv(save_file_name)
    print("File: ", i, "\tTable: ", tbl_flag, "Row: ", row_flag)



  3%|▎         | 1/33 [00:04<02:33,  4.81s/it]

File:  0 	Table:  1 Row:  1



  6%|▌         | 2/33 [00:08<02:16,  4.40s/it]

File:  1 	Table:  1 Row:  1



  9%|▉         | 3/33 [00:10<01:52,  3.75s/it]

...

File:  28 	Table:  1 Row:  1
File:  29 	Table:  1 Row:  1
File:  30 	Table:  1 Row:  1



 97%|█████████▋| 32/33 [00:50<00:01,  1.62s/it]

File:  31 	Table:  1 Row:  1



100%|██████████| 33/33 [00:53<00:00,  2.02s/it]

File:  32 	Table:  1 Row:  1


In [9]:
#results_df = pd.DataFrame(results)
results_df = pd.read_csv(save_file_name)
row_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 1)] # table but no row
none_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 0)] # no table no row
last_check = results_df[(results_df['row'] == 1) & (results_df['tbl'] == 1)] # table and row, other issue?
print("row & table found: ", len(set(last_check['fname'])))
print("table found: ", len(set(row_check['fname'])))
print("nothing found: ", len(set(none_check['fname'])))

row & table found:  7
table found:  0
nothing found:  0


In [10]:
last_check

,1-3,3-5,<1,>5,category,cik,datadate,fname,multiplier,row,tbl,total
0,2708.3,1018.7,1719.7,28.0,purchase obligations(a),0000009389,20031231,0000009389-04-000059.txt,1000000,1,1,5474.7
1,3204.9,1128.3,1832.1,1113.1,total payments on contractual obligations,0000009389,20031231,0000009389-04-000059.txt,1000000,1,1,7278.4
2,1,0,98,0,and construction of buildings,0000003333,20040129,0000003333-04-000014.txt,1,1,1,99.0
3,0,0,80,0,transportation contracts,0000003333,20040129,0000003333-04-000014.txt,1,1,1,80.0
4,1346,831,1480,7532,total contractual cash obligations,0000003333,20040129,0000003333-04-000014.txt,1,1,1,11189.0
5,77,49,63,31,capital lease obligations,0000022198,20031231,0000004904-04-000060.txt,1000000,1,1,220.0
6,2132,1101,1720,1785,obligations (a),0000022198,20031231,0000004904-04-000060.txt,1000000,1,1,6738.0
7,492,441,291,2331,noncancellable operating leases,0000022198,20031231,0000004904-04-000060.txt,1000000,1,1,3555.0
8,2512,0,2512,7200,other long0term obligations (b),0000800459,20040630,0000800459-04-000047.txt,1000,1,1,12224.0
9,0,0,20674,000,firm commitments for capital expenditures,0000800459,20040630,0000800459-04-000047.txt,1000,1,1,20674.0


# Examining one file

In [116]:
test_f = open(path_prefix + "0001457143-09-000008.txt")
wholetext = test_f.read()
cik_re = get_cik(wholetext)
datadate_re = get_datadate(wholetext)  
print("CIK: ", cik_re, "Date: ", datadate_re)
wholetext = wholetext.lower()

# Text table debugging
#text_chunk, multiplier, lines, tbl_list = scrape_text_table(wholetext)
# mult, lines, tbl_list = scrape_text_table(wholetext)
# rows = scrape_text_rows(lines, tbl_list)
# HTML debugging 
#scores, tables = scrape_table(wholetext)
# table = scrape_table(wholetext)
# rows = scrape_rows(table)
# mult = get_multiplier_from_tbl(table)
rows, mult

CIK:  0001277151 Date:  20081231


([], 1)

In [114]:
table_to_list(table)

[['for the period\nnovember 9, 2007\n\xa0 (initial capitalization)\nthrough\ndecember 31, 2007',
  'year ended\ndecember 31,\n2008',
  'revenues: room revenue other revenue total revenue expenses: operating expense hotel administrative expense sales and marketing utilities repair and maintenance franchise fees management fees taxes, insurance and other general and administrative depreciation expense total expenses operating loss interest income interest expense net income (loss) basic and diluted net income (loss) per common share',
  'weighted average common shares outstanding -\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 basic and diluted',
  '15,852',
  'distributions declared and paid per common share'],
 ['-',
  '-',
  '- $',
  '2,023',
  '11,524',
  '-',
  '3,381',
  '883',
  '1,066',
  '546',
  '637',
  '468',
  '441',
  '731',
  '1,288',
  '2,277',
  '11,718',
  '(194',
  '2,721',
  '(375',
  '$ $',
  '-',
  '- $'],
 ['-',
  '-',
  '-',
  '9,501',
  '-',
  '-',
  '-',
  '-',
  '-',
 

In [74]:
tbl_list

[['contractual obligations and commercial commitments. at december',
  '31,',
  '2008,',
  'our'],
 ['after', '2009', '2010', '2011', '2012', '2013', '2013', 'total'],
 ['operating leases', '606', '607', '474', '55', '14', '--', '1,756'],
 ['purchase', 'obligations', '4,852', '--', '--', '--', '--', '--', '4,852'],
 ['-------',
  '------',
  '------',
  '------',
  '------',
  '------',
  '------- total',
  '$6,075',
  '$1,223',
  '$1,090',
  '671',
  '630',
  '$3,489',
  '$13,178'],
 ['',
  'our purchase obligations are amounts committed under legally enforceable'],
 ['short duration of the securities in which we invest and the conservativenature of our investment portfolio, a',
  '10%',
  'move in interest rates over a'],
 ['one-year period would have an immaterial effect of approximately',
  '$20,000',
  'of'],
 ['decreased interest income on our financial position, results of operationsand cash flows. if the yen strengthens against the dollar by',
  '10%,',
  'the'],
 ['have an imm

In [75]:
text_chunk
l = text_chunk.split('\n')
numbers_row_indices = [hasNumbers(x) for x in l]
start_row_idx = np.where(np.array(numbers_row_indices) == True)[0][0]
end_row_idx = np.where(np.array(numbers_row_indices) == True)[0][-1]

In [52]:
l[start_row_idx:end_row_idx+1]

['                   payments due by period at december 31, 2008',
 '                                 (in thousands)',
 '',
 '                                            less than     one to       three to     more than',
 'contractual obligations           total      one year   three years   five years   five years',
 '- -----------------------           -----     ---------   -----------   ----------   ----------',
 '<s>                             <c>         <c>         <c>           <c>          <c>',
 'certificates of deposit and     $ 143,190   $ 110,433   $    25,548   $    5,574   $    1,635',
 '  other time deposits',
 'short-term debt',
 '  obligations                       3,911       3,911             -            -            -',
 'minimum operating lease',
 '  obligations                       1,745         817           849           63           16',
 '                                ---------   ---------   -----------   ----------   ----------',
 '       total         

In [56]:
lines = l[start_row_idx:end_row_idx+1]
line_no = 0
condensed_lines = []
while line_no < len(lines):
    if hasNumbers(lines[line_no]):
        if line_no > 1 and not hasNumbers(lines[line_no-1]):
            condensed_lines.append(lines[line_no-1] + lines[line_no])
        else:
            condensed_lines.append(lines[line_no])
    line_no += 1
condensed_lines

['                   payments due by period at december 31, 2008',
 '<s>                             <c>         <c>         <c>           <c>          <c>certificates of deposit and     $ 143,190   $ 110,433   $    25,548   $    5,574   $    1,635',
 'short-term debt  obligations                       3,911       3,911             -            -            -',
 'minimum operating lease  obligations                       1,745         817           849           63           16',
 '                                ---------   ---------   -----------   ----------   ----------       total                    $ 148,846   $ 115,161   $    26,397   $    5,637   $    1,651',
 "sensitivity  models and monitor  earnings at risk. these limits are based on thebank's  exposure to a 100 bp and 200 bp immediate  and  sustained  parallel rate",
 '                                       24']

In [34]:
lines = condensed_lines
keywords = HDR_KWS + ROW_KWS
lines = [x for x in lines if (hasKeywords(x, keywords))]
lines = [x.split(' ') for x in lines]
for i, line in enumerate(lines):
    lines[i] = list(filter(lambda a: (a != '') and (a != '$') 
                           and (a !='<u>') and (a !='</u>'), line))    

In [67]:
tbl_list = table_to_list(table)
tbl_list

[['long-term\n      debt', '200.0', '-', '-', '-', '200.0'],
 ['loans\n      from\n      affiliates', '1,325.0', '-', '-', '-', '1,325.0'],
 ['operating\n      leases', '3,356.9', '203.7', '394.0', '403.1', '2,356.1'],
 ['employee\n      benefits', '37.0', '2.5', '6.4', '7.3', '20.8'],
 ['total\n      contractual obligations $',
  '91,511.1',
  '$',
  '2,643.7',
  '$',
  '4,888.0',
  '$',
  '4,954.1',
  '$',
  '79,025.3']]

In [74]:
n_prev_siblings = 6
preceding_text = ' '.join([str(x) for x in itertools.islice(table.previous_siblings, n_prev_siblings)])

In [7]:
node = table
el_list = []
for i in range(20):
    el_list.append(node.parent)
    node = node.parent

In [38]:
prev_text = table.findAllPrevious(text=True)[:500]
prev_text = [x.replace('\xa0', '') for x in prev_text]
prev_text = [x.replace('\n', '') for x in prev_text]
prev_text = [x.replace('$', '') for x in prev_text]
prev_text = [x.replace(' ', '') for x in prev_text]
prev_text = [x for x in prev_text if ('' != x)][:30]
prev_text = ' '.join(prev_text)

In [39]:
get_multiplier_from_string(prev_text)

1000000